## FastAPI webservice and deploy a model
FastAPI is used to create a webservice 'app' to accept HTTP requests.

MySentimentModel class loads the ML model and defines *predict* function for online inference. @serve.deployment decorator defines the Ray Serve deployment.

*@app.get()* is used to create a GET '/predict' route. Similarly, @app.post() can be used POST requests. See https://docs.ray.io/en/latest/serve/http-guide.html for more details.

In this example, *application_logic()* function is used to define a sample transformation or business logic that can be applied before sending the input to the ML model for inference. See inline comments for further explanation.

### Scaling deployment
*num_replicas* parameter sets the number of instances of the deployment. FastAPI and RayServe automatically load balances to send requests to each instance. There are more options to set the *accelerator_type* to GPU and even use fractional GPUs. See configuration options here: https://docs.ray.io/en/latest/serve/configure-serve-deployment.html .

In [2]:

# Define a simple FastAPI app
app = FastAPI()

# Define a Ray Serve deployment
# This decorator registers the class as a Ray Serve deployment
@serve.deployment(num_replicas=2) # num_replicas specifies the number of replicas for load balancing
@serve.ingress(app) # This decorator allows the FastAPI app to be served by Ray Serve
class MySentimentModel:
    def __init__(self):
        # Load a pre-trained sentiment analysis model
        self.model = pipeline("sentiment-analysis",
                              model="distilbert-base-uncased-finetuned-sst-2-english")

    # Define any necessary application logic or transformation logic
    def application_logic(self, text):
        """        Apply any necessary application logic to the input text.
        """
        # simple application logic: truncate text if it exceeds a certain length
        if len(text) > 50:
            return text[:50].lower()  # Truncate and convert to lowercase
        else:
            return text.lower()
        
    @app.get("/predict") # Define an endpoint for predictions
    def predict(self, text: str):
        """        Predict sentiment for the given text.
        """
        # Define any necessary application logic or transformation logic
        text = self.application_logic(text) # Apply any necessary application logic to the input text

        # Use the model to make a prediction
        result = self.model(text)
        return {"text": text, "sentiment": result}
 




 ### Deploy the model

In [3]:

serve.run(MySentimentModel.bind()) # Bind the deployment to the Ray Serve runtime


2025-07-11 09:16:58,624	INFO worker.py:1908 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(ProxyActor pid=56603) INFO 2025-07-11 09:16:59,927 proxy 127.0.0.1 -- Proxy starting on node c2986ec5ee9361515a52f3c506843d5d247ffc6732250203b8a47f21 (HTTP port: 8000).
(ProxyActor pid=56603) INFO 2025-07-11 09:16:59,946 proxy 127.0.0.1 -- Got updated endpoints: {}.
INFO 2025-07-11 09:16:59,975 serve 18071 -- Started Serve in namespace "serve".
(ServeController pid=56591) INFO 2025-07-11 09:17:00,075 controller 56591 -- Deploying new version of Deployment(name='MySentimentModel', app='default') (initial target replicas: 2).
(ServeController pid=56591) INFO 2025-07-11 09:17:00,178 controller 56591 -- Adding 2 replicas to Deployment(name='MySentimentModel', app='default').
(ProxyActor pid=56603) INFO 2025-07-11 09:17:00,076 proxy 127.0.0.1 -- Got updated endpoints: {Deployment(name='MySentimentModel', app='default'): EndpointInfo(route='/', app_is_cross_language=Fals

DeploymentHandle(deployment='MySentimentModel')

(ServeReplica:default:MySentimentModel pid=56599) INFO 2025-07-11 09:20:39,996 default_MySentimentModel kcd1ofsq 1d920c68-1c59-4e2e-a64c-04f87d31bab7 -- GET /predict 200 128.0ms
(ServeReplica:default:MySentimentModel pid=56595) INFO 2025-07-11 09:21:11,350 default_MySentimentModel t4o4t2pu dda6dadc-f201-421d-9bbc-638d92b9f065 -- GET /predict 200 543.2ms
(ServeReplica:default:MySentimentModel pid=56599) INFO 2025-07-11 09:21:15,969 default_MySentimentModel kcd1ofsq e3611603-54d1-491a-8b05-1f693f413243 -- GET /predict 200 36.5ms
(ServeReplica:default:MySentimentModel pid=56599) INFO 2025-07-11 09:22:16,285 default_MySentimentModel kcd1ofsq 5be0e806-0bd5-4992-b15d-c59fd52e6195 -- GET /predict 200 422.7ms
(ServeReplica:default:MySentimentModel pid=56599) INFO 2025-07-11 09:22:58,953 default_MySentimentModel kcd1ofsq 8890c548-abf6-49b2-b923-b525fcc403da -- GET /predict 200 76.5ms
(ServeController pid=56591) INFO 2025-07-11 09:23:20,248 controller 56591 -- Removing 2 replicas from Deployment